# Codebook

## bd CPS list of variables and possible values

April 29, 2019

Brian Dew, @bd_econ, brianwdew@gmail.com

-----

Store a list of bd CPS variables and their datatype, availability, average coverage, and list of possible entries.

In [1]:
# Preliminaries
import pandas as pd
import numpy as np
import sys
old_stdout = sys.stdout
sys.stdout = open('/home/brian/Documents/econ_data/bd_CPS/codebook.txt', 'w')

import os
os.chdir('/home/brian/Documents/econ_data/bd_CPS/')

from bd_CPS_details import CodebookNotes, ValueLabels

os.chdir('/home/brian/Documents/CPS/data/clean/')

/home/brian/Documents/econ_data/bd_CPS/bd_CPS_details.py:67: SyntaxWarning: invalid escape sequence '\s'
  're': f'({"|".join([x for x in VarList if x not in LostVars])})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'},
/home/brian/Documents/econ_data/bd_CPS/bd_CPS_details.py:70: SyntaxWarning: invalid escape sequence '\s'
  're': f'({"|".join(VarList)})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'},
/home/brian/Documents/econ_data/bd_CPS/bd_CPS_details.py:73: SyntaxWarning: invalid escape sequence '\s'
  're': f'({"|".join([x for x in VarList if x not in LostVars])})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'},
/home/brian/Documents/econ_data/bd_CPS/bd_CPS_details.py:76: SyntaxWarning: invalid escape sequence '\s'
  're': f'({"|".join(VarList)})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'},
/home/brian/Documents/econ_data/bd_CPS/bd_CPS_details.py:79: SyntaxWarning: invalid escape sequence '\s'
  're': f'({"|".join(VarList)})\s+(\d+)\s+.*?\t+.*?(\d\d*).*?(\d\d+)'},
/home/brian/Documents/econ_data/bd_CPS/bd_CPS_d



    bd CPS Codebook


updated: January 24, 2026

variables:

MONTH - Month
    Notes: Survey reference month. Original CPS variable: HRMONTH
    Data types: ['int8']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 100.0%
    Value range: 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]



FAMINC - Family Income category
    Notes: Total family income in bin. Additional categories added starting October 2003. Variabled edited starting January 2010. Many instances are allocated by BLS/Census.
    Data types: ['category']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 92.6% (negative values treated as missing)
    Value range: 
[-3, -2, -1, 1, 2, 3]  ...  [11, 12, 13, 14, 15, 16]



MIS - Month in sample
    Notes: Household month in sample. Original CPS variable: HRMIS
    Data types: ['int8']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 100.0%
    Value range: 
[1, 2, 3, 4, 5, 6, 7, 8]



HHWGT - Household weight
    Notes: Household weight, derived 

In [2]:
# Go through data and store requested details (optimized)
d = {}

def clean_value(v):
    """Convert numpy types to native Python types"""
    if hasattr(v, 'item'):
        return v.item()
    return v

def has_negative_values(col):
    """Check if column has any negative values (handles categorical and numeric)"""
    try:
        # For numeric columns
        return (col < 0).any()
    except TypeError:
        # For categorical columns, check the categories or unique values
        try:
            vals = col.dropna().unique()
            return any(v < 0 for v in vals if isinstance(v, (int, float)))
        except:
            return False

def get_valid_mask(col, has_neg):
    """Get mask for valid (non-missing) values"""
    if not has_neg:
        return col.notna()
    try:
        # For numeric columns
        return col.notna() & (col >= 0)
    except TypeError:
        # For categorical columns, exclude negative values by checking each
        vals = col.dropna().unique()
        neg_vals = [v for v in vals if isinstance(v, (int, float)) and v < 0]
        if neg_vals:
            mask = col.notna()
            for nv in neg_vals:
                mask = mask & (col != nv)
            return mask
        return col.notna()

for year in range(1989, 2026):
    data = pd.read_feather(f'cps{year}.ft')
    variables = list(data.columns)
    
    # Get months present in this year's data
    months = sorted(data['MONTH'].unique())
    
    # Pre-compute which variables have data in which months (vectorized)
    # This replaces the slow per-variable month-by-month filtering
    month_col = data['MONTH']
    var_has_data_by_month = data.notna().groupby(month_col).any()
    
    for var in variables:
        col = data[var]
        col_notna = col.dropna()
        
        # Skip if all null or only one unique value across entire year
        if len(col_notna) == 0:
            continue
        values_unique = col_notna.unique()
        if len(values_unique) <= 1:
            continue
            
        dtype_name = col.dtype.name
        
        # Check if this variable uses negative values as missing
        has_neg = has_negative_values(col)
        
        # Calculate valid count for allocation rate
        # Valid = not NaN and (>= 0 if variable uses negative values as missing)
        valid_mask = get_valid_mask(col, has_neg)
        valid_count = valid_mask.sum()
        total_count = len(col)
        
        if var not in d:
            # Find first/last months with data using pre-computed boolean DataFrame
            months_with_data = var_has_data_by_month[var]
            valid_months = months_with_data[months_with_data].index
            if len(valid_months) > 0:
                first_month = valid_months.min()
                last_month = valid_months.max()
            else:
                continue  # No valid months, skip
            
            d[var] = {
                'breaks': [],
                'date_min': pd.to_datetime(f'{year}-{first_month}-01'),
                'date_max': pd.to_datetime(f'{year}-{first_month}-01'),
                'dtype': [dtype_name],
                'values': set(),
                'max_val': None,
                'min_val': None,
                'valid_count': valid_count,
                'total_count': total_count,
                'uses_neg': has_neg
            }
        else:
            # Check for dtype change
            if dtype_name not in d[var]['dtype']:
                d[var]['breaks'].append(year)
                d[var]['dtype'].append(dtype_name)
            
            # Accumulate allocation rate counts
            d[var]['valid_count'] += valid_count
            d[var]['total_count'] += total_count
            if has_neg:
                d[var]['uses_neg'] = True
        
        # Update last month with data using pre-computed boolean DataFrame
        months_with_data = var_has_data_by_month[var]
        valid_months = months_with_data[months_with_data].index
        if len(valid_months) > 0:
            last_month = valid_months.max()
            d[var]['date_max'] = pd.to_datetime(f'{year}-{last_month}-01')
        
        # Collect values (use set for O(1) lookups)
        n_unique = len(values_unique)
        if n_unique < 100 and 'HHID2' not in var and 'OTC' not in var and 'PRICE' not in var:
            # Clean numpy types to native Python types
            d[var]['values'].update(clean_value(v) for v in values_unique)
        else:
            # Track min/max for variables with many unique values
            try:
                loc_min = clean_value(values_unique.min())
                loc_max = clean_value(values_unique.max())
            except TypeError:
                sorted_vals = sorted(values_unique)
                loc_min = clean_value(sorted_vals[0])
                loc_max = clean_value(sorted_vals[-1])
            if d[var]['min_val'] is None or loc_min < d[var]['min_val']:
                d[var]['min_val'] = loc_min
            if d[var]['max_val'] is None or loc_max > d[var]['max_val']:
                d[var]['max_val'] = loc_max

# Format dates, convert sets to sorted lists, and calculate allocation rates
for key, values in d.items():
    # Convert values set to sorted list
    try:
        values['values'] = sorted(values['values'])
    except TypeError:
        # Mixed types that can't be sorted together
        values['values'] = sorted(values['values'], key=str)
    
    # Format dates as YYYY-MM strings
    values['date_min'] = values['date_min'].strftime('%Y-%m')
    values['date_max'] = values['date_max'].strftime('%Y-%m')
    
    # Calculate allocation rate
    values['alloc_rate'] = values['valid_count'] / values['total_count']

## Print Codebook

In [3]:
# Print out the bd CPS codebook

# Variables derived/created by bd_CPS (not original CPS variables)
DerivedVars = {
    'WBHAO', 'WBAO', 'WBHAOM',  # Race recodes
    'INDM', 'INDGRP',  # Industry recodes
    'CPSID', 'HHID2', 'QSTNUM', 'OCCURNUM',  # ID variables
    'WKEARNADJ', 'HRWAGEADJ',  # Adjusted earnings
    'HRSUSL1I',  # Imputed hours
    'EDUC', 'EDUCDT', 'HGCI',  # Education recodes
    'LFS', 'EMP', 'FTLF', 'PTECON',  # Labor force status
    'REGION',  # From STATE
    'FEMALE',  # From PESEX
    'MARRIED',  # From PRMARSTA
    'FORBORN', 'CITIZEN',  # From PRCITSHP
    'HISPANIC',  # From PRORIGIN/PRDTHSP
    'VETERAN',  # From PEAFEVER
    'METSTAT', 'MPCSTAT',  # Metro status recodes
    'WKEARN', 'HRWAGE',  # Earnings (converted from raw)
    'HHWGT', 'BASICWGT', 'PWORWGT',  # Weights (converted)
    'STATE',  # From GESTFIPS
    'NILFREASON', 'NLFFAM', 'UNEMPTYPE',  # NILF/unemployment recodes
    'MINWAGE',  # Minimum wage flag
    'SCHENR', 'SCHOOL',  # School enrollment
    'USLFT',  # Full-time status
    'NCHILDU5', 'NCHILDU18',  # Child counts
    'MANAGER',  # Occupation flag
    'PRICEADJ',  # Price adjustment factor
    'OTCAMT',  # Overtime/tips/commissions
    'TLWK', 'TLWKHR',  # Telework (from supplement)
}

def truncate_values(vals, max_show=12):
    """Truncate long value lists, showing first and last few items"""
    if len(vals) <= max_show:
        return str(vals)
    half = max_show // 2
    first = vals[:half]
    last = vals[-half:]
    return f'{first}  ...  {last}'

print('\n========================\n\n    bd CPS Codebook'
      '\n\n========================\n')
today = pd.to_datetime('today').strftime('%B %d, %Y')
print(f'updated: {today}\n\nvariables:\n')

for key, values in d.items():
    # Variable name and description
    if key in CodebookNotes.keys():
        print(f'{key} - {CodebookNotes[key]["Name"]}')
        print(f'    Notes: {CodebookNotes[key]["Notes"]}')
    else:
        print(f'{key}')
    
    # Derived flag
    if key in DerivedVars:
        print('    Source: Derived by bd_CPS')
    
    print(f'    Data types: {values["dtype"]}')
    print(f'    Available from: {values["date_min"]} to: {values["date_max"]}')
    
    # Allocation rate (percent of observations with valid data)
    alloc_pct = values['alloc_rate'] * 100
    neg_note = ' (negative values treated as missing)' if values.get('uses_neg', False) else ''
    print(f'    Allocation rate: {alloc_pct:.1f}%{neg_note}')
    
    if len(values['breaks']) > 0:
        print(f'    Breaks in dtype: {values["breaks"]}')
    
    # Value range with truncation for long lists
    print('    Value range: ')
    if len(values['values']) > 0:
        print(truncate_values(values['values']))
    else:
        print(f'{values["min_val"]} to {values["max_val"]}')
    print('\n\n')

print('\n==============================\n\n    Variable value labels'
      '\n\n==============================\n\n')

print('Imputed highest grade completed (HGCI):\n')
for key, value in ValueLabels['HGCI'].items():
    print(f'{key}     {value}')

print('\n\n2018 Occupation codes (OCC18):\n')
for key, value in ValueLabels['OCC18'].items():
    print(f'{key}     {value}')
              
print('\n\n2010 Occupation detailed recodes (OCC03D):\n')
for key, value in ValueLabels['OCC03D'].items():
    print(f'{key}     {value}')
              
print('\n\n2010 Occupation major recodes (OCC03M):\n')
for key, value in ValueLabels['OCC03M'].items():
    print(f'{key}     {value}')
              
print('\n\n\n2017 Industry codes (IND17):\n')
for key, value in ValueLabels['IND17'].items():
    print(f'{key}     {value}')
              
print('\n\n2012 Industry detailed recodes (IND03D):\n')
for key, value in ValueLabels['IND03D'].items():
    print(f'{key}     {value}')
              
print('\n\n2012 Industry major recodes (IND03M):\n')
for key, value in ValueLabels['IND03M'].items():
    print(f'{key}     {value}')
              
print('\n\nCore-based statistical areas (CBSA):\n')
for key, value in ValueLabels['CBSA'].items():
    print(f'{key}     {value}')
              
print('\n\nConsolidated statistical areas (CSA):\n')
for key, value in ValueLabels['CSA'].items():
    print(f'{key}     {value}')
              
print('\n\nCounties (COUNTY):\n')
for key, value in ValueLabels['COUNTY'].items():
    print(f'{key}     {value}')

In [4]:
sys.stdout = old_stdout

codebook = open('/home/brian/Documents/econ_data/bd_CPS/codebook.txt', 'r').read()
print(codebook)

In [5]:
print(codebook)



    bd CPS Codebook


updated: January 24, 2026

variables:

MONTH - Month
    Notes: Survey reference month. Original CPS variable: HRMONTH
    Data types: ['int8']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 100.0%
    Value range: 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]



FAMINC - Family Income category
    Notes: Total family income in bin. Additional categories added starting October 2003. Variabled edited starting January 2010. Many instances are allocated by BLS/Census.
    Data types: ['category']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 92.6% (negative values treated as missing)
    Value range: 
[-3, -2, -1, 1, 2, 3]  ...  [11, 12, 13, 14, 15, 16]



MIS - Month in sample
    Notes: Household month in sample. Original CPS variable: HRMIS
    Data types: ['int8']
    Available from: 1989-01 to: 2025-12
    Allocation rate: 100.0%
    Value range: 
[1, 2, 3, 4, 5, 6, 7, 8]



HHWGT - Household weight
    Notes: Household weight, derived 